In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from neural_network.trainer import Agent
# Note: Assume the following: (batch_size, channel, height, width, depth)

ModuleNotFoundError: No module named 'neural_network.configs'

In [ ]:
agent = Agent('resnet50')

In [ ]:
#agent.fit()

In [ ]:
iter(agent.dataset.train_dataloader()).next()[0].shape

In [ ]:
#agent.model

In [ ]:
device = 'cuda'
input_tensor = iter(agent.dataset.val_dataloader()).next()[0][0].unsqueeze(1).to(device)

In [ ]:
input_tensor.device

In [ ]:
#model = agent.model.to(device)

In [ ]:
from torchvision.models import resnet18
from torchcam.cams import SSCAM
import torch

def get_CAM(agent,input_tensor):
    model = agent.model.to(device)
    model.eval()
    
    cam = SSCAM(
        model.model, 
        'layer4',
        batch_size = 1,
        num_samples = 1,
        std = 2.0,
        input_shape = (1, 79, 224, 224),
    )
    with torch.no_grad(): 
        scores = model.model(input_tensor)
        activation_map = cam(scores.squeeze(0).argmax().item(), scores).cpu()
    return activation_map,scores
get_CAM(agent, input_tensor)

In [ ]:
from skimage.transform import resize
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import math

In [ ]:
input_tensor.shape

In [ ]:
imgmap = cm.get_cmap('gray')
img = imgmap(input_tensor.cpu().squeeze(0).squeeze(0))[:,:,:,1:]

In [ ]:
plt.imshow(img[69])

In [ ]:
show_slices(img)

In [ ]:
def show_slices(data):
    f,axes= plt.subplots(7,math.ceil(len(data[0][0])/7), figsize=(14,12.5),subplot_kw={'xticks': [], 'yticks': []})
    f.subplots_adjust(hspace=0.01, wspace=0.01)
    ax = axes.flat
    for i in range(len(data[0][0])):
        ax[i].imshow(data[:,:,i],vmin=0,vmax=255)

In [ ]:

cmap = cm.get_cmap('jet')
imgmap = cm.get_cmap('gray')
alpha=0.7

def show_slices(data):
    f,axes= plt.subplots(7,math.ceil(len(data[0][0])/7), figsize=(14,12.5),subplot_kw={'xticks': [], 'yticks': []})
    f.subplots_adjust(hspace=0.01, wspace=0.01)
    ax = axes.flat
    for i in range(len(data[0][0])):
        ax[i].imshow(data[:,:,i],vmin=0,vmax=255)

#for mask,img in zip(heatmap, input_tensor.cpu().squeeze(0).squeeze(0)):
"""
fig = plt.figure()
overlay = (255 * cmap(np.asarray(mask) ** 2)[:, :, 1:]).astype(np.uint8)
# Overlay the image with the mask

img = imgmap(img)[:,:,1:]# = img.unsqueeze(0)
#img = img.numpy().transpose((1,2,0))
#print(img.shape, mask.shape, overlay.shape, np.asarray(img).shape)
overlayed_img = (alpha * np.asarray(img) + (1 - alpha) * overlay).astype(np.uint8)
#print(overlayed_img.shape)
plt.imshow(mask)
"""
heatmap = resize(activation_map, (79,95,79))
heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
heatmap = np.uint8(255 * heatmap)

#overlay = np.uint8(255 * heatmap)
jet_colors = cmap(np.arange(256))[:, :3]
overlay = jet_colors[heatmap]

#overlay = (255*cmap(np.asarray(heatmap))[:, :, :,1:]).astype(np.uint8)
img = imgmap(input_tensor.cpu().squeeze(0).squeeze(0))[:,:,:,1:]

#print(overlay.max(), img.max(),((1-alpha) * overlay).min())
print(img.shape,overlay.shape)
overlayed_img = (alpha * np.asarray(img) + (1 - alpha) * overlay)
#print(alpha * np.asarray(img))
show_slices(overlayed_img)